In [1]:
import csv
import requests
import json
import argparse
import traceback
import pandas as pd
import numpy as np
from time import sleep
import ast

# Using pre-cleaned data-set
Source: https://www.kaggle.com/kerneler/starter-mcdonald-s-locations-2678722e-c

In [2]:
data = pd.read_csv("data/McDonalds.csv")
data = data.groupby("properties.subDivision").count()
data = data[[data.columns[0]]]
data.index = ["us-" + i.lower() for i in data.index]
data["State"] = [i for i in data.index]
data["JS Format"] = [[data["State"].iloc[j], data["geometry.coordinates"].iloc[j]] for j in range(len(data))]
data["JS Format"].to_csv("data/state_locations_count_data.csv", index=False)

# Scraping for our own data
Source: https://www.yelp.com/fusion

In [3]:
data = 'data/cities_by_state.json'
with open(data) as json_file:
    cities = json.load(json_file)

df = pd.DataFrame.from_dict(cities, orient='index')
cities_lists = df.to_numpy()

In [4]:
# Create data dictionary for state (key) and cities (value)
data_dict = {}
for i in range(len(df)):
    data_dict[df.index[i]] = [city for city in cities_lists[i] if type(city) == str]

scrape_list = []
for state in data_dict.keys():
    scrape_list += data_dict[state]

In [5]:
def get_locations(locations, term, auth, url, offset_range=5):
    """
    User-defined function to retrieve business locations.
    
    :param locations: A list of locations to retrieve from.
    :param term: A particular business to retrieve.
    :param category: A word associated with the term.
    :param auth: An api key or code required to make requests.
    :param url: URL to retrieve from.
    :param offset_range: Number of request to make per city.
    """
    api_key = auth
    headers = {'Authorization': 'Bearer %s' % api_key}
    url = url
    df_list = []
    num_requests = 0
    
    for i in locations:
        for j in range(offset_range):
            num_requests += 1
            params = {"term": term, "location": i, "radius": 40000, "limit": 50, "offset": j*50}
            req = requests.get(url, params=params, headers=headers, timeout=30)
            # print('The status code for offset={} {} is {} for term={}'.format(j, i, req.status_code, term))
            if req.status_code == 200:
                res_json = json.loads(req.text)["businesses"]
                curr_df = pd.DataFrame(res_json)
                df_list.append(curr_df)
        sleep(3)
            
    print(num_requests)
    return df_list

In [6]:
# To avoid issues with too many requests, we'll split the list into 4 and run each separately
api_key = 'eWYw0VZjlZbbiSmZttZna1D6RzIXnUvayhoqvyRRHgpcsJM2pB7hNg1Xo4DOGFtO0LTz8i_xJsLGv_ytzUTF8kVaqkxeDKf8OGeYKLy0geuhv_ERXYuRvpU9t73eXnYx'
url = 'https://api.yelp.com/v3/businesses/search'

locations_df = get_locations(locations=scrape_list[split], 
                             term="McDonald's", 
                             auth=api_key,
                             url=url)

In [7]:
combined_df = pd.concat(locations_df)
state_codes = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
               'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
               'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
               'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
               'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

res_df = combined_df.drop_duplicates(subset=["id"])
res_df = res_df[res_df["is_closed"]==False]
res_df = res_df[res_df["alias"].str.contains("mcd")]
res_states = [res_df["location"].iloc[i]["state"] for i in range(res_df.shape[0])]
res_df["State"] = res_states
res_df = res_df[res_df["State"].isin(state_codes)]

res_df = res_df.groupby("State").count()[["id"]].astype(float)
res_df.index = ["us-" + i.lower() for i in res_df.index]

# Format for javascript
res_df["State"] = [i for i in res_df.index]
res_df["JS Format"] = [[res_df["State"].iloc[j], res_df["id"].iloc[j]] for j in range(len(res_df))]
res_df["JS Format"].to_csv("data/state_locations_count.csv", index=False)

In [8]:
yelp_js = pd.read_csv("data/state_locations_count.csv")

# Choosing representative data
It's difficult to get the exact numbers for locations because of how large the franchise is (stores open and close all the time). Upon examining the data from the Kaggle source and the data we retrieved through Yelp's API, we found that some states had underestimates in both data-set (via a Google search). For example, the Kaggle data-sets values were mostly larger (and closer to the values from Google searching, but were missing Hawaii and underestimated Alaska). To get the most precise numbers possible, we took both data-sets and saved the max value between the two.

In [9]:
# Reformatting the list pairs from string to list
yelp_js["JS Format"] = [ast.literal_eval(i) for i in yelp_js["JS Format"]]

In [10]:
data.loc["us-hi"] = [0, "us-hi", ["us-hi", 0]]
data = data.sort_index()
data_js = data[["JS Format"]].reset_index(drop=True)

In [11]:
imputed_data = pd.DataFrame()
imputed_data["State"] = [i[0] for i in yelp_js["JS Format"]]
imputed_data["Count_Yelp"] = [i[1] for i in yelp_js["JS Format"]]
imputed_data["Count_Data"] = [i[1] for i in data_js["JS Format"]]
imputed_data["Count"] = imputed_data[["Count_Yelp", "Count_Data"]].max(axis=1)
imputed_data["Result"] = [[imputed_data["State"].iloc[i], imputed_data["Count"].iloc[i]] for i in range(imputed_data.shape[0])]
imputed_data["Result"].to_csv("data/map_data.csv")

In [12]:
pd.read_csv("data/map_data.csv")["Result"]

0       ['us-ak', 18.0]
1      ['us-al', 245.0]
2      ['us-ar', 173.0]
3      ['us-az', 289.0]
4     ['us-ca', 1288.0]
5      ['us-co', 211.0]
6      ['us-ct', 147.0]
7       ['us-dc', 28.0]
8       ['us-de', 37.0]
9      ['us-fl', 907.0]
10     ['us-ga', 454.0]
11      ['us-hi', 53.0]
12     ['us-ia', 147.0]
13      ['us-id', 62.0]
14     ['us-il', 663.0]
15     ['us-in', 349.0]
16     ['us-ks', 148.0]
17     ['us-ky', 251.0]
18     ['us-la', 240.0]
19     ['us-ma', 242.0]
20     ['us-md', 288.0]
21      ['us-me', 62.0]
22     ['us-mi', 545.0]
23     ['us-mn', 226.0]
24     ['us-mo', 315.0]
25     ['us-ms', 144.0]
26      ['us-mt', 48.0]
27     ['us-nc', 489.0]
28      ['us-nd', 25.0]
29      ['us-ne', 78.0]
30      ['us-nh', 54.0]
31     ['us-nj', 264.0]
32     ['us-nm', 104.0]
33     ['us-nv', 144.0]
34     ['us-ny', 617.0]
35     ['us-oh', 615.0]
36     ['us-ok', 204.0]
37     ['us-or', 164.0]
38     ['us-pa', 494.0]
39      ['us-ri', 32.0]
40     ['us-sc', 230.0]
41      ['us-sd'

In [13]:
# Number of Stores in the world by Year
# Source: https://expandedramblings.com/index.php/mcdonalds-statistics/
count_by_year = [{"Year": 1994, "Total": 15899}, 
                 {"Year": 1995, "Total": 18299}, 
                 {"Year": 1996, "Total": 20884}, 
                 {"Year": 1997, "Total": 22928}, 
                 {"Year": 1998, "Total": 24513}, 
                 {"Year": 1999, "Total": 26309}, 
                 {"Year": 2000, "Total": 28707}, 
                 {"Year": 2001, "Total": 30093}, 
                 {"Year": 2002, "Total": 31108}, 
                 {"Year": 2003, "Total": 31129}, 
                 {"Year": 2004, "Total": 31561}, 
                 {"Year": 2005, "Total": 31886}, 
                 {"Year": 2006, "Total": 31046}, 
                 {"Year": 2007, "Total": 31377}, 
                 {"Year": 2008, "Total": 31967}, 
                 {"Year": 2009, "Total": 32478}, 
                 {"Year": 2010, "Total": 32737}, 
                 {"Year": 2011, "Total": 33510}, 
                 {"Year": 2012, "Total": 34480}, 
                 {"Year": 2013, "Total": 35429}, 
                 {"Year": 2014, "Total": 36258}, 
                 {"Year": 2015, "Total": 36525}, 
                 {"Year": 2016, "Total": 36899}, 
                 {"Year": 2017, "Total": 37241}, 
                 {"Year": 2018, "Total": 37855}]

count_json = json.dumps(count_by_year)

# with open("data/year_counts.json", "w") as outfile: 
#    outfile.write(count_json)

# Nutrition Data

In [14]:
df = pd.read_csv('data/menu.csv')
mcd_menu = pd.read_csv('data/menu.csv')

In [15]:
mcd_menu.head()

,Category,Item,Serving Size,Calories,Calories from Fat,Total Fat,Total Fat (% Daily Value),Saturated Fat,Saturated Fat (% Daily Value),Trans Fat,...,Carbohydrates,Carbohydrates (% Daily Value),Dietary Fiber,Dietary Fiber (% Daily Value),Sugars,Protein,Vitamin A (% Daily Value),Vitamin C (% Daily Value),Calcium (% Daily Value),Iron (% Daily Value)
0,Breakfast,Egg McMuffin,4.8 oz (136 g),300,120,13.0,20,5.0,25,0.0,...,31,10,4,17,3,17,10,0,25,15
1,Breakfast,Egg White Delight,4.8 oz (135 g),250,70,8.0,12,3.0,15,0.0,...,30,10,4,17,3,18,6,0,25,8
2,Breakfast,Sausage McMuffin,3.9 oz (111 g),370,200,23.0,35,8.0,42,0.0,...,29,10,4,17,2,14,8,0,25,10
3,Breakfast,Sausage McMuffin with Egg,5.7 oz (161 g),450,250,28.0,43,10.0,52,0.0,...,30,10,4,17,2,21,15,0,30,15
4,Breakfast,Sausage McMuffin with Egg Whites,5.7 oz (161 g),400,210,23.0,35,8.0,42,0.0,...,30,10,4,17,2,21,6,0,25,10


In [16]:
mcd_menu.Item.values

array(['Egg McMuffin', 'Egg White Delight', 'Sausage McMuffin',
       'Sausage McMuffin with Egg', 'Sausage McMuffin with Egg Whites',
       'Steak & Egg McMuffin',
       'Bacon, Egg & Cheese Biscuit (Regular Biscuit)',
       'Bacon, Egg & Cheese Biscuit (Large Biscuit)',
       'Bacon, Egg & Cheese Biscuit with Egg Whites (Regular Biscuit)',
       'Bacon, Egg & Cheese Biscuit with Egg Whites (Large Biscuit)',
       'Sausage Biscuit (Regular Biscuit)',
       'Sausage Biscuit (Large Biscuit)',
       'Sausage Biscuit with Egg (Regular Biscuit)',
       'Sausage Biscuit with Egg (Large Biscuit)',
       'Sausage Biscuit with Egg Whites (Regular Biscuit)',
       'Sausage Biscuit with Egg Whites (Large Biscuit)',
       'Southern Style Chicken Biscuit (Regular Biscuit)',
       'Southern Style Chicken Biscuit (Large Biscuit)',
       'Steak & Egg Biscuit (Regular Biscuit)',
       'Bacon, Egg & Cheese McGriddles',
       'Bacon, Egg & Cheese McGriddles with Egg Whites',
       'Sau

In [17]:
list(mcd_menu.columns)

['Category',
 'Item',
 'Serving Size',
 'Calories',
 'Calories from Fat',
 'Total Fat',
 'Total Fat (% Daily Value)',
 'Saturated Fat',
 'Saturated Fat (% Daily Value)',
 'Trans Fat',
 'Cholesterol',
 'Cholesterol (% Daily Value)',
 'Sodium',
 'Sodium (% Daily Value)',
 'Carbohydrates',
 'Carbohydrates (% Daily Value)',
 'Dietary Fiber',
 'Dietary Fiber (% Daily Value)',
 'Sugars',
 'Protein',
 'Vitamin A (% Daily Value)',
 'Vitamin C (% Daily Value)',
 'Calcium (% Daily Value)',
 'Iron (% Daily Value)']

In [18]:
# https://www.cheatsheet.com/culture/the-top-10-best-selling-mcdonalds-menu-items-of-all-time.html/

# URL above is list of mcd top 10 items
# Top 5 Items
#1 = french fries
#2 = big mac
#3 = snack wraps

In [19]:
num3 = mcd_menu[mcd_menu['Item'].str.contains('Snack Wrap')]
num2 = mcd_menu[mcd_menu['Item'].str.contains('Big Mac')]
num1 = mcd_menu[mcd_menu['Item'].str.contains('Fries')]
num5 = mcd_menu[mcd_menu['Item'].str.contains('Egg McMuffin')]

In [20]:
top3 = pd.concat([num1, num2, num3])

In [21]:
desired_data = top3[['Item', 'Calories', 'Total Fat', 'Total Fat (% Daily Value)', 'Sodium', 'Sodium (% Daily Value)', 'Sugars']]
desired_data.index = desired_data['Item']

In [22]:
desired_data = desired_data[['Calories', 'Total Fat', 'Total Fat (% Daily Value)', 'Sodium', 'Sodium (% Daily Value)', 'Sugars']]
desired_data

,Calories,Total Fat,Total Fat (% Daily Value),Sodium,Sodium (% Daily Value),Sugars
Item,,,,,,
Small French Fries,230,11.0,17,130,5,0
Medium French Fries,340,16.0,24,190,8,0
Large French Fries,510,24.0,37,290,12,0
Kids French Fries,110,5.0,8,65,3,0
Big Mac,530,27.0,42,960,40,9
Chipotle BBQ Snack Wrap (Crispy Chicken),340,15.0,23,780,33,8
Chipotle BBQ Snack Wrap (Grilled Chicken),260,8.0,13,700,29,7
Honey Mustard Snack Wrap (Crispy Chicken),330,15.0,23,730,30,3
Honey Mustard Snack Wrap (Grilled Chicken),250,8.0,13,650,27,2


In [23]:
mcd_menu.groupby("Category").count()["Item"].sort_values().values

array([ 6,  7, 13, 15, 27, 27, 28, 42, 95], dtype=int64)

In [24]:
mcd_menu.groupby("Category").count()["Item"].sort_values().index

Index(['Salads', 'Desserts', 'Snacks & Sides', 'Beef & Pork', 'Beverages',
       'Chicken & Fish', 'Smoothies & Shakes', 'Breakfast', 'Coffee & Tea'],
      dtype='object', name='Category')